In [1]:
import pandas as pd
import os
import requests
from io import BytesIO
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import tensorflow as tf

from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from PIL import ImageFile
from collections import Counter
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from imblearn.over_sampling import SMOTE
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

import gc

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
def load_image(row, local_path_dataset, target_size):
    """
    Cargar una imagen la cual puede estar local o remotar en una url
    Argments
        row:                 Registro panda con la información de un ejemplo de entrada: la imagen puede estar local o remota (urlAbsoluta)
        local_path_dataset:  Ruta local donde se encuentran las imágens
        target_size:         Tamñao de la imagen de salia una vez cargada
    Returns
        Imagen redimensionada y como un arreglo
    """
    #Cargar el csv para encontrar el nombre o Url de los archivos a cargas
    try:
        if pd.notna(row['urlAbsoluta']):
            # Descargar la imagen desde la URL
            response = requests.get(row['urlAbsoluta'], timeout=5)
            if response.status_code == 200:
                image = load_img(BytesIO(response.content), target_size=target_size)
            else:
                return None
        else:
            # Cargar la imagen localmente
            # ajuste según dataset local
            filename = row['filename']
            prefix = 'no_dogs' if "nodogs" in filename else 'dogs'
            directory = os.path.join(local_path_dataset, prefix)
            local_path = os.path.join(directory, filename)
            if os.path.exists(local_path):
                image = load_img(local_path, target_size=target_size)
            else:
                return None

            
        return img_to_array(image) / 255.0  # Escalar al rango [0, 1]
        
    except requests.exceptions.RequestException as e:
            print(f"Error al descargar la imagen {row['urlAbsoluta']}: {e}")
            return None    
    except FileNotFoundError:
            print(f"Archivo no encontrado: {row['filename']}")
            return None
    except Exception as e:
            print(f"Error al abrir la imagen {row['filename']}: {e}")
            return None

In [3]:
def preprocess_labels(row):
    """
    Procesar las etiquetas como un vector binario
    Argments
        row:                 Registro panda con la información de un ejemplo de entrada: la imagen puede estar local o remota (urlAbsoluta)
    Returns
        arreglo binario con las etiqutas
    """
    #return [row['direccion'], row['fachada'], row['envio'], row['etiqueta']]
    return [row['perro'], row['gato']]

In [4]:
def prepare_input(row, local_path_dataset, target_size): 
    """
    Prepare los datos basado en el registro del archivo csv de entrada
    Arguments
        row:                 Registro panda con la información de un ejemplo de entrada: la imagen puede estar local o remota (urlAbsoluta)
        local_path_dataset:  Ruta local donde se encuentran las imágens
        target_size:         Tamñao de la imagen de salia una vez cargada
    Returns 
        [arreglo con las imagenes, arreglo con los labels]
    """
     # Preparar las etiquetas
    labels = preprocess_labels(row)
    img_array = load_image(row, local_path_dataset, target_size)

    return img_array, np.array(labels)
    

In [5]:
def read_input_data(csv_path, local_path_dataset, target_size, max_workers=4):
    df = pd.read_csv(csv_path)

    # Inicializar array para almacenar imágenes
    images = []
    # Inicializar array para almacenar labels
    labels = []

    label_columns = ['perro', 'gato']
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
         # Procesar cada fila del DataFrame
         futures = [executor.submit(prepare_input, row, local_path_dataset, target_size) 
                    for _, row in df.iterrows()]
         
         for future in tqdm(futures, total=len(df)):
             result = future.result()
             if result is not None:
                 img_array, img_labels = result

                 if img_array is not None and img_array.shape == (224, 224, 3):  # Verificar forma
                     images.append(img_array)
                     labels.append(img_labels)
                 else:
                      print(f"Imagen inválida o con tamaño incorrecto")

    X = np.array(images)
    y = np.array(labels)
        
    print(f"\nDataset preparado con {len(X)} imágenes")
    print(f"Distribución de clases:")
    for i, col in enumerate(label_columns):
        positive_samples = np.sum(y[:, i])
        percentage = (positive_samples / len(y)) * 100
        print(f"{col}: {percentage:.2f}% ({int(positive_samples)}/{len(y)})")
        
    return X, y

In [6]:
def multi_label_smote(X, y, k_neighbors=5):
    """
    Aplica SMOTE a un dataset multi-label.
    Genera datos sintéticos basados en las combinaciones de etiquetas.
    """
    # Combinar etiquetas multi-label en cadenas únicas para identificarlas
    label_combinations = [''.join(map(str, labels)) for labels in y]
    
    # Convertir etiquetas en índices únicos
    unique_combinations, y_indices = np.unique(label_combinations, return_inverse=True)
    
    # Aplicar SMOTE sobre las combinaciones
    smote = SMOTE(k_neighbors=k_neighbors)
    
    # Aplanar las imágenes a vectores unidimensionales
    X_flattened = X.reshape(len(X), -1)
    X_resampled_flattened, y_resampled_indices = smote.fit_resample(X_flattened, y_indices)

    # Restaurar la forma original de las imágenes
    X_resampled = X_resampled_flattened.reshape(-1, 224, 224, 3)
    
    # Convertir los índices resampleados de nuevo a multi-label
    y_resampled = np.array([list(map(int, comb)) for comb in unique_combinations[y_resampled_indices]])
    
    return X_resampled, y_resampled


In [7]:
# Función para calcular factores de aumentación
def calculate_augmentation_factors(labels, target_balance=0.25):
    num_samples = len(labels)
    label_counts = np.sum(labels, axis=0)
    max_samples = target_balance * num_samples
    augmentation_factors = {i: int(np.ceil(max_samples / count)) if count > 0 else 1
                             for i, count in enumerate(label_counts)}
    return augmentation_factors

In [8]:
def balanced_batch_generator_with_smote(X, y, batch_size, smote_frequency=10, k_neighbors=5):
    """
    Generador de datos balanceados usando la técnica de SMOTE


    Returns
        X,y conjuntos balanceados para hacer el entrenamiento
    """
    # Dividir los índices de las clases
    from collections import defaultdict
    class_indices = defaultdict(list)

    for i, labels in enumerate(y):
        for cls, label in enumerate(labels):
            if label == 1:  # Etiqueta positiva
                class_indices[cls].append(i)

    label_columns = ['perro', 'gato']
    n_classes = y.shape[1]
    counter = 0  # Para rastrear cuándo aplicar SMOTE

    # Crear un batch balanceado
    while True:
        batch_X, batch_y = [], []

        # Aplicar SMOTE cada cierto número de batches
        if counter % smote_frequency == 0:
            X, y = multi_label_smote(X, y, k_neighbors=k_neighbors)

        # Crear un batch balanceado
        for _ in range(batch_size):
            # Seleccionar aleatoriamente una clase
            cls = np.random.choice(n_classes)
            
            # Obtener índices aleatorios de esa clase
            sample_idx = np.random.choice(class_indices[cls])

            # Agregar al batch
            batch_X.append(X[sample_idx])
            batch_y.append(y[sample_idx])

        counter += 1
        X_array = np.array(batch_X)
        y_array = np.array(batch_y)

        print(f"\nDataset preparado con {len(X_array)} imágenes")
        print(f"Distribución de clases:")
        for i, col in enumerate(label_columns):
            positive_samples = np.sum(y_array[:, i])
            percentage = (positive_samples / len(y_array)) * 100
            print(counter)
            print(f"{col}: {percentage:.2f}% ({int(positive_samples)}/{len(y_array)})")
            
        yield X_array, y_array

In [9]:
def build_model(target_size, output_units=2):
    # Carga del modelo base
    base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=target_size)

    # Congela capas iniciales
    base_model.trainable = False

    # Agrega capas personalizadas
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dense(output_units, activation='sigmoid')  # Activación para multi-label
    ])

    # Compila el modelo
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy', AUC(name='auc')])

    return model

In [10]:
# Aumentación adaptativa
def augment_images_adaptive(X, y, augmentation_factors, datagen):
    augmented_images = []
    augmented_labels = []

    for i in range(len(X)):
        image = X[i]
        label = y[i]
        max_factor = max([augmentation_factors[j] for j, present in enumerate(label) if present])

        image = np.expand_dims(image, axis=0)
        augmented_iter = datagen.flow(image, batch_size=1)
        
        for _ in range(max_factor):
            augmented_images.append(next(augmented_iter)[0])
            augmented_labels.append(label)

    return np.array(augmented_images), np.array(augmented_labels)

In [11]:
# Balanceo con SMOTE
def balance_with_smote(X, y):
    smote = SMOTE(random_state=42)
    X_flattened = X.reshape(len(X), -1)
    y_encoded = encode_multilabel(y)
    X_resampled_flattened, y_resampled_encoded = smote.fit_resample(X_flattened, y_encoded)
    X_resampled = X_resampled_flattened.reshape(-1, 224, 224, 3)
    y_resampled = decode_multilabel(y_resampled_encoded)
    
    return X_resampled, y_resampled

In [12]:
def decode_multilabel(encoded_labels, num_classes=4):
    return np.array([[int(char) for char in label] for label in encoded_labels])

In [13]:
# Convertir etiquetas multietiqueta
def encode_multilabel(labels):
    return np.array(["".join(map(str, label)) for label in labels])


In [14]:
def visualize_images(images, labels, title, num_images=5):
    """
    Muestra un conjunto de imágenes con sus etiquetas asociadas.
    
    Args:
        images (np.array): Arreglo de imágenes (formato HWC, RGB).
        labels (np.array): Etiquetas asociadas a las imágenes.
        title (str): Título de la visualización.
        num_images (int): Número de imágenes a mostrar.
    """
    plt.figure(figsize=(15, 5))
    for i in range(min(num_images, len(images))):
        plt.subplot(1, num_images, i + 1)
        img = images[i]
        
        # Asegurar que la imagen esté en el rango [0, 255]
        if img.max() <= 1.0:
            img = img * 255.0
        img = img.astype("uint8")
        
        plt.imshow(img)
        plt.title(f"Etiqueta: {labels[i]}")
        plt.axis("off")
    plt.suptitle(title)
    plt.show()


In [15]:
def plot_training_history(history):
    """
    Muestra los gráficos de pérdida y precisión durante el entrenamiento.
    
    Args:
        history (History): Objeto de historial devuelto por model.fit().
    """
    # Extraer métricas
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    accuracy = history.history.get('accuracy', [])
    val_accuracy = history.history.get('val_accuracy', [])

    epochs = range(1, len(loss) + 1)

    # Gráfico de pérdida
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, 'bo-', label='Pérdida Entrenamiento')
    plt.plot(epochs, val_loss, 'ro-', label='Pérdida Validación')
    plt.title('Pérdida durante el Entrenamiento')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.legend()

    # Gráfico de precisión (si está disponible)
    if accuracy:
        plt.subplot(1, 2, 2)
        plt.plot(epochs, accuracy, 'bo-', label='Precisión Entrenamiento')
        plt.plot(epochs, val_accuracy, 'ro-', label='Precisión Validación')
        plt.title('Precisión durante el Entrenamiento')
        plt.xlabel('Épocas')
        plt.ylabel('Precisión')
        plt.legend()

    plt.tight_layout()
    plt.show()


In [16]:
IMG_SIZE = (224, 224)
INPUT_SIZE = (224, 224, 3)
BATCH_SIZE = 8
DATASET_PATH = "../DataSets/cats_vs_dogs/"

csv_path = './dogs-no-dogs.csv'

print('inicio')
X, y = read_input_data(csv_path, local_path_dataset=DATASET_PATH, target_size=IMG_SIZE, max_workers=4)
print('read_input_data')
print('X=', X.shape)
print('y=', y.shape)

np.savez_compressed('X_y', X=X, y=y)
print('savez_compressed')

#loaded = np.load('X_y.npz')
#X = loaded['X']
#y = loaded['y']
#print('X=', X.shape)
#print('y=', y.shape)

# Configuración de Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Cargar datos (se supone que tienes las variables `images` y `labels`)
# Divide en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print('train_test_split')
del X
del y
gc.collect()
visualize_images(X_train[:5], y_train[:5], "Imágenes Originales")

# Aumentación adaptativa
augmentation_factors = calculate_augmentation_factors(y_train, target_balance=0.25)
print('calculate_augmentation_factors')
X_train_augmented, y_train_augmented = augment_images_adaptive(X_train, y_train, augmentation_factors, datagen)
visualize_images(X_train_augmented[:5], y_train_augmented[:5], "Imágenes Después de Aumentación")

print('augment_images_adaptive')
print("Distribución después de aumentación:", Counter(encode_multilabel(y_train_augmented)))

del X_train
del y_train
gc.collect()

# Aplicar SMOTE
X_train_balanced, y_train_balanced = balance_with_smote(X_train_augmented, y_train_augmented)
visualize_images(X_train_balanced[:5], y_train_balanced[:5], "Imágenes Después de SMOTE")

print('balance_with_smote')
print("Distribución después de SMOTE:", Counter(encode_multilabel(y_train_balanced)))

del X_train_augmented
del y_train_augmented
gc.collect()


model = build_model(target_size=INPUT_SIZE, output_units=2)
print('build_model')


# Checkpoint para guardar el mejor modelo
checkpoint = ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", mode="min")
# Entrenamiento del modelo
history = model.fit(
    X_train_balanced, y_train_balanced,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=BATCH_SIZE,
    callbacks=[checkpoint]
)
# Entrenar el modelo
#model.fit(generator, steps_per_epoch=len(X) // BATCH_SIZE, epochs=20)
print('fit')

# Guardar el modelo
model.save("dog_nodogs_balanced.keras", save_format="keras")

# Visualizar el historial de entrenamiento
plot_training_history(history)


inicio


  0%|                                                                                | 4/24999 [00:00<07:43, 53.93it/s]


NameError: name 'row' is not defined